# March Madness Notebook

In [1]:
import Pkg
using Gurobi
using JuMP
using LinearAlgebra
using Random
using Statistics

In [2]:
using Serialization

n_teams = 64
max_round = 6

# When using synthetic probabilities
P_synthetic = deserialize("Prob_synthetic.dat")

# When using real probabilities 
P_real = deserialize("Prob_real.dat")


64×64 Matrix{Float64}:
 0.5    0.998  0.794  0.758  0.782  …  0.966  0.755  0.862  0.606  0.981
 0.002  0.5    0.022  0.017  0.02      0.159  0.016  0.041  0.005  0.227
 0.206  0.978  0.5    0.452  0.484     0.844  0.448  0.606  0.291  0.896
 0.242  0.983  0.548  0.5    0.532     0.871  0.496  0.652  0.334  0.916
 0.218  0.98   0.516  0.468  0.5       0.853  0.464  0.622  0.305  0.903
 0.109  0.945  0.341  0.298  0.326  …  0.726  0.295  0.444  0.169  0.802
 0.255  0.985  0.564  0.516  0.548     0.879  0.512  0.666  0.348  0.922
 0.085  0.928  0.291  0.251  0.278     0.677  0.248  0.39   0.136  0.761
 0.218  0.98   0.516  0.468  0.5       0.853  0.464  0.622  0.305  0.903
 0.245  0.984  0.552  0.504  0.536     0.873  0.5    0.655  0.337  0.918
 ⋮                                  ⋱         ⋮                    
 0.066  0.907  0.245  0.209  0.233  …  0.626  0.206  0.337  0.107  0.716
 0.261  0.986  0.571  0.524  0.556     0.883  0.52   0.674  0.356  0.925
 0.176  0.971  0.456  0.409  0.44

In [ ]:
function compute_T(n_teams=64)
    T = Dict{Tuple{Int,Int}, Set{Int}}()
    # Base case: Round 1 (32 games)
    for k in 1:32
        T[(k,1)] = Set([2k-1, 2k])
    end
    # Recursive case: Rounds 2-6
    for r in 2:6
        games_in_round = 64 ÷ (2^(r-1))
        for k in 1:games_in_round
            prev_game1 = 2k-1
            prev_game2 = 2k
            # Only add if the previous games exist
            if haskey(T, (prev_game1, r-1)) && haskey(T, (prev_game2, r-1))
                T[(k,r)] = union(T[(prev_game1, r-1)], T[(prev_game2, r-1)])
            end
        end
    end
    return T
end


function compute_E(T)
    E = Dict{Tuple{Int,Int}, Set{Int}}()
    for i in 1:n_teams
        for r in 1:max_round
            if r == 1
                E[(i,r)] = Set()
            else
                k_current = ceil(Int, i / (2^r))
                k_prev = ceil(Int, i / (2^(r-1)))
                current_teams = T[(k_current, r)]
                prev_teams = T[(k_prev, r-1)]
                E[(i,r)] = setdiff(current_teams, prev_teams)
            end
        end
    end
    return E
end


T = compute_T()
E = compute_E(T)


Dict{Tuple{Int64, Int64}, Set{Int64}} with 384 entries:
  (23, 2) => Set([22, 21])
  (41, 2) => Set([43, 44])
  (54, 6) => Set([5, 16, 20, 30, 19, 32, 6, 9, 31, 29  …  14, 3, 7, 25, 15, 2, …
  (49, 4) => Set([64, 61, 58, 60, 57, 59, 63, 62])
  (63, 6) => Set([5, 16, 20, 30, 19, 32, 6, 9, 31, 29  …  14, 3, 7, 25, 15, 2, …
  (56, 5) => Set([35, 37, 47, 41, 43, 45, 36, 44, 39, 33, 46, 40, 48, 34, 38, 4…
  (35, 5) => Set([56, 55, 58, 52, 60, 53, 49, 51, 64, 61, 57, 59, 50, 54, 63, 6…
  (62, 6) => Set([5, 16, 20, 30, 19, 32, 6, 9, 31, 29  …  14, 3, 7, 25, 15, 2, …
  (34, 1) => Set()
  (50, 1) => Set()
  (60, 5) => Set([35, 37, 47, 41, 43, 45, 36, 44, 39, 33, 46, 40, 48, 34, 38, 4…
  (58, 6) => Set([5, 16, 20, 30, 19, 32, 6, 9, 31, 29  …  14, 3, 7, 25, 15, 2, …
  (42, 1) => Set()
  (46, 3) => Set([41, 43, 44, 42])
  (59, 2) => Set([58, 57])
  (57, 3) => Set([64, 61, 63, 62])
  (26, 2) => Set([27, 28])
  (21, 4) => Set([29, 25, 30, 28, 32, 27, 26, 31])
  (38, 4) => Set([46, 48, 47, 41, 43, 45

In [4]:
function calculate_Wir(P, E)
    n_teams = size(P, 1)
    max_round = 6
    W = Dict{Tuple{Int,Int}, Float64}()

    # Base case: Round 1
    for i in 1:n_teams
        opponent = iseven(i) ? i-1 : i+1
        W[(i,1)] = P[i, opponent]
    end

    # Recursive case: Rounds 2-6
    for r in 2:max_round
        for i in 1:n_teams
            total = 0.0
            for j in E[(i,r)]
                # Probability j reaches round r-1 and i beats j
                total += W[(j,r-1)] * P[i,j]
            end
            W[(i,r)] = W[(i,r-1)] * total
        end
    end
    W
end

W_real = calculate_Wir(P_real, E)
W_synthetic = calculate_Wir(P_synthetic, E)

Dict{Tuple{Int64, Int64}, Float64} with 384 entries:
  (23, 2) => 0.0690167
  (41, 2) => 0.430398
  (54, 6) => 0.00496313
  (49, 4) => 0.0512531
  (63, 6) => 0.00774466
  (56, 5) => 0.144834
  (35, 5) => 0.0233333
  (62, 6) => 0.0148893
  (34, 1) => 0.229069
  (50, 1) => 0.402069
  (60, 5) => 0.0250732
  (58, 6) => 0.0044392
  (42, 1) => 0.390024
  (46, 3) => 0.195136
  (59, 2) => 0.0982099
  (57, 3) => 0.363611
  (26, 2) => 0.210632
  (21, 4) => 0.0363389
  (38, 4) => 0.0482444
  ⋮       => ⋮

In [5]:
function optimal_bracket(W, n_teams=64)
    model = Model(Gurobi.Optimizer)

    @variable(model, x[i=1:n_teams, r=1:6], Bin)

    @objective(model, Max, sum(2^(r-1) * W[(i, r)] * x[i, r] for i in 1:n_teams, r in 1:6))

    # Constraint 1: If team i wins in round r, it must win in all previous rounds
    for i in 1:n_teams
        for r in 2:6
            @constraint(model, x[i, r] <= x[i, r-1])
        end
    end

    # Constraint 2: In each round r, exactly 2^(6-r) teams win
    for r in 1:6
        @constraint(model, sum(x[i, r] for i in 1:n_teams) == 2^(6-r))
    end

    # Constraint 3: In the first round, either team 2k-1 or team 2k wins (not both)
    for k in 1:32
        @constraint(model, x[2*k-1, 1] + x[2*k, 1] == 1)
    end

    # Constraint 4: In subsequent rounds, only one team from each game in the previous round can win
    T = compute_T(n_teams)
    for r in 2:6
        games_prev_round = 2^(7-r)
        for k in 1:games_prev_round
            teams_in_game = T[(k, r-1)]
            @constraint(model, sum(x[i, r] for i in teams_in_game) <= 1)
        end
    end

    optimize!(model)

    if termination_status(model) == MOI.OPTIMAL
        bracket = Dict()
        for r in 1:6
            bracket[r] = []
            for i in 1:n_teams
                if value(x[i, r]) > 0.5
                    push!(bracket[r], i)
                end
            end
        end
        expected_score = objective_value(model)
        return bracket, expected_score
    else
        return nothing, 0
    end
end


optimal_bracket (generic function with 2 methods)

In [ ]:
println("\nFinding optimal bracket with BPI probabilities...")
bracket_real, score_real = optimal_bracket(W_real)
println("Optimal bracket with BPI probabilities:")
for r in 1:6
    println("Round $r: $(sort(bracket_real[r]))")
end
println("Expected score: $score_real")


Finding optimal bracket with BPI probabilities...
Set parameter Username
Set parameter LicenseID to value 2618922
Academic license - for non-commercial use only - expires 2026-02-06
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 420 rows, 384 columns and 1408 nonzeros
Model fingerprint: 0xd9ac3174
Variable types: 0 continuous, 384 integer (384 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-11, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 27.3186647
Presolve removed 99 rows and 64 columns
Presolve time: 0.01s
Presolved: 321 rows, 320 columns, 1088 nonzeros
Found heuristic solution: objective 31.6726647
Variable types: 0 continuous, 320 integer (320 binary)

Root relaxation: objective 9.338174e+01, 85 iterations, 0.00 seconds 

In [7]:
println("\nFinding optimal bracket with synthetic probabilities...")
bracket_synthetic, score_synthetic = optimal_bracket(W_synthetic)
println("Optimal bracket with synthetic probabilities:")
for r in 1:6
    println("Round $r: $(sort(bracket_synthetic[r]))")
end
println("Expected score: $score_synthetic")


Finding optimal bracket with synthetic probabilities...
Set parameter Username
Set parameter LicenseID to value 2618922
Academic license - for non-commercial use only - expires 2026-02-06
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 420 rows, 384 columns and 1408 nonzeros
Model fingerprint: 0x4f533860
Variable types: 0 continuous, 384 integer (384 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 23.8139732
Presolve removed 99 rows and 64 columns
Presolve time: 0.00s
Presolved: 321 rows, 320 columns, 1088 nonzeros
Found heuristic solution: objective 26.7719150
Variable types: 0 continuous, 320 integer (320 binary)

Root relaxation: objective 6.026444e+01, 77 iterations, 0.00 se

In [ ]:
function greedy_bracket(P::Array{Float64,2})
    n_teams = size(P, 1)
    bracket = Dict{Int, Vector{Int}}()
    teams_remaining = collect(1:n_teams)

    for r in 1:6
        winners = Int[]
        n_games = length(teams_remaining) ÷ 2
        for k in 1:n_games
            i = teams_remaining[2k-1]
            j = teams_remaining[2k]
            if P[i, j] > 0.5
                push!(winners, i)
            elseif P[i, j] < 0.5
                push!(winners, j)
            else
                push!(winners, min(i, j))
            end
        end
        bracket[r] = winners
        teams_remaining = winners
    end
    return bracket
end

bracket_real = greedy_bracket(P_real)
bracket_synthetic = greedy_bracket(P_synthetic)

Dict{Int64, Vector{Int64}} with 6 entries:
  5 => [20, 56]
  4 => [3, 20, 33, 56]
  6 => [56]
  2 => [3, 5, 11, 15, 20, 21, 28, 32, 33, 39, 41, 48, 49, 56, 60, 64]
  3 => [3, 11, 20, 28, 33, 41, 56, 64]
  1 => [2, 3, 5, 7, 10, 11, 14, 15, 18, 20  …  46, 48, 49, 51, 53, 56, 57, 60, …

In [9]:
println("Greedy bracket with BPI probabilities:")
for r in 1:6
    println("Round $r: $(sort(bracket_real[r]))")
end

Greedy bracket with BPI probabilities:
Round 1: [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63]
Round 2: [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63]
Round 3: [1, 11, 17, 27, 33, 47, 49, 63]
Round 4: [1, 17, 33, 49]
Round 5: [1, 33]
Round 6: [33]


In [10]:
println("\nGreedy bracket with synthetic probabilities:")
for r in 1:6
    println("Round $r: $(sort(bracket_synthetic[r]))")
end


Greedy bracket with synthetic probabilities:
Round 1: [2, 3, 5, 7, 10, 11, 14, 15, 18, 20, 21, 24, 26, 28, 30, 32, 33, 36, 38, 39, 41, 44, 46, 48, 49, 51, 53, 56, 57, 60, 61, 64]
Round 2: [3, 5, 11, 15, 20, 21, 28, 32, 33, 39, 41, 48, 49, 56, 60, 64]
Round 3: [3, 11, 20, 28, 33, 41, 56, 64]
Round 4: [3, 20, 33, 56]
Round 5: [20, 56]
Round 6: [56]


In [11]:
"""
Simulate a single game between two teams based on their win probability
"""
function simulate_game(team1::Int, team2::Int, P::Matrix{Float64})
    if rand() < P[team1, team2]
        return team1
    else
        return team2
    end
end

"""
Simulate a round of the tournament given a list of matchups
"""
function simulate_round(matchups::Vector{Tuple{Int,Int}}, P::Matrix{Float64})
    winners = Int[]
    for (team1, team2) in matchups
        winner = simulate_game(team1, team2, P)
        push!(winners, winner)
    end
    return winners
end

"""
Create initial matchups for the first round in sequential pairs (1-2, 3-4, etc.)
"""
function create_initial_matchups(n_teams::Int)
    matchups = Tuple{Int,Int}[]
    for i in 1:2:n_teams
        push!(matchups, (i, i+1))
    end
    return matchups
end

"""
Create matchups for next round based on previous round's winners
"""
function create_next_matchups(winners::Vector{Int})
    n_winners = length(winners)
    matchups = Tuple{Int,Int}[]
    
    for i in 1:2:n_winners
        push!(matchups, (winners[i], winners[i+1]))
    end
    
    return matchups
end

"""
Simulate entire tournament and return results for each round
"""
function simulate_tournament(n_teams::Int, P::Matrix{Float64})
    # Use a Vector to maintain order of rounds
    results = Vector{Vector{Int}}()
    
    # Initial matchups
    matchups = create_initial_matchups(n_teams)
    
    #Uncomment this for printing the winners

    #println("Initial matchups:")
    #for (i, matchup) in enumerate(matchups)
    #    println("Game $i: Team $(matchup[1]) vs Team $(matchup[2])")
    #end

    println()
    
    # Simulate each round
    round_num = 1
    while length(matchups) > 0
        winners = simulate_round(matchups, P)
        push!(results, winners)  # Add winners to results vector
        
        #println("Round $round_num winners: $winners")
        
        if length(winners) == 1
            # println("\nTournament Winner: Team $(winners[1])")
            break
        end
        
        matchups = create_next_matchups(winners)
        round_num += 1
    end
    
    return results
end

# Example usage:

simulate_tournament

In [18]:
# Run tournament simulation
tournament_results = simulate_tournament(n_teams, P_real)

6-element Vector{Vector{Int64}}:
 [1, 4, 6, 7, 10, 11, 13, 15, 17, 20  …  46, 47, 49, 52, 53, 55, 58, 59, 61, 63]
 [1, 7, 11, 13, 20, 22, 27, 31, 33, 39, 42, 47, 49, 55, 59, 63]
 [1, 11, 20, 27, 33, 47, 55, 59]
 [11, 27, 33, 55]
 [11, 33]
 [33]

In [19]:
# Calculating the score of the results between predicted and true results 

function calculate_score(results_predicted::Vector{Vector{Int}}, results_true::Vector{Vector{Int}})
    points_per_round = [1, 2, 4, 8, 16, 32]  # Points for each round
    
    total_score = 0
    
    for (round, (pred, actual)) in enumerate(zip(results_predicted, results_true))
        # Count matching predictions in this round
        correct_picks = sum(p == t for (p, t) in zip(pred, actual))
        round_points = correct_picks * points_per_round[round]
        
        #println("Round $round: $correct_picks correct predictions × $(points_per_round[round]) points = $round_points points")
        total_score += round_points
    end
    
    #println("\nTotal Score: $total_score points")
    return total_score
end


println(calculate_score(tournament_results, tournament_results))

192


In [20]:
function if_valid(bracket::Vector{Vector{Int}}, n_teams::Int)

    not_valid = 0

    num_round = Int(log2(n_teams))

    if num_round != length(bracket)
        not_valid = 1
    end
    
    for i in 1:num_round
        # Check if the length of the current round matches the expected number of teams
        if length(bracket[i]) != n_teams ÷ (2^i)
            not_valid = 1
        end
    
        if i == 1
            # Validate the first round
            for j in 1:length(bracket[i])
                if bracket[i][j] != 2*j-1 && bracket[i][j] != 2*j
                    not_valid = 1
                end
            end
        else
            # Validate subsequent rounds
            for j in 1:length(bracket[i])
                if bracket[i][j] != bracket[i-1][2*j-1] && bracket[i][j] != bracket[i-1][2*j]
                    not_valid = 1
                end
            end
        end
    end

    return not_valid

end


if_valid (generic function with 1 method)

In [21]:
# Convert dictionary format to vector of vectors format for easier use
function convert_bracket_format(bracket_dict)
    bracket_vector = Vector{Vector{Int}}()
    for r in 1:6
        push!(bracket_vector, sort(bracket_dict[r]))
    end
    return bracket_vector
end

optimal_bracket_vector = convert_bracket_format(bracket_real)

6-element Vector{Vector{Int64}}:
 [1, 4, 5, 7, 10, 11, 13, 15, 17, 19  …  45, 47, 49, 51, 53, 55, 57, 59, 61, 63]
 [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63]
 [1, 11, 17, 27, 33, 47, 49, 63]
 [1, 17, 33, 49]
 [1, 33]
 [33]

In [ ]:
is_valid(bracket, n_teams) = valid(bracket, n_teams) == 0

# Check if two brackets are identical
function are_brackets_identical(br1::Vector{Vector{Int}}, br2::Vector{Vector{Int}})
    length(br1) == length(br2) && all(br1[i] == br2[i] for i in 1:length(br1))
end

# Check uniqueness of a bracket in a group
function is_unique(bracket::Vector{Vector{Int}}, group::Vector{Vector{Vector{Int}}})
    for br in group
        if are_brackets_identical(bracket, br)
            return false
        end
    end
    return true
end


is_unique (generic function with 1 method)

In [ ]:
using JSON

# Save to JSON file
file_path = "March_madness.json"
open(file_path, "w") do file
    JSON.print(file, brackets)
end

println("Brackets saved to $file_path")


Brackets saved to team_Group8_10.json


In [25]:
function mutate_bracket(bracket::Vector{Vector{Int}}, n_teams::Int; max_attempts=10)
    for attempt in 1:max_attempts
        new_bracket = deepcopy(bracket)
        rounds = length(bracket)
        r = rand(2:rounds)
        picks = new_bracket[r]
        prev_round = new_bracket[r-1]
        idx = rand(1:length(picks))
        valid_choices = setdiff(prev_round, [picks[idx]])
        if !isempty(valid_choices)
            picks[idx] = rand(valid_choices)
            new_bracket[r] = picks
            for rr in r+1:rounds
                prev_winners = new_bracket[rr-1]
                current = new_bracket[rr]
                new_picks = [p in prev_winners ? p : rand(prev_winners) for p in current]
                new_bracket[rr] = new_picks
            end
            if is_valid(new_bracket, n_teams)
                return new_bracket
            end
        end
    end
    return bracket
end

function crossover_brackets(parent1::Vector{Vector{Int}}, parent2::Vector{Vector{Int}}, n_teams::Int)
    rounds = length(parent1)
    point = rand(2:rounds)
    child1 = vcat(parent1[1:point-1], parent2[point:end])
    child2 = vcat(parent2[1:point-1], parent1[point:end])
    for child in (child1, child2)
        for r in point+1:rounds
            prev_winners = child[r-1]
            current = child[r]
            new_picks = [p in prev_winners ? p : rand(prev_winners) for p in current]
            child[r] = new_picks
        end
    end
    child1 = is_valid(child1, n_teams) ? child1 : parent1
    child2 = is_valid(child2, n_teams) ? child2 : parent2
    return child1, child2
end


crossover_brackets (generic function with 1 method)

In [26]:
function initialize_group(base_bracket::Vector{Vector{Int}}, n_teams::Int, group_size::Int=10)
    group = Vector{Vector{Vector{Int}}}()
    push!(group, base_bracket)
    while length(group) < group_size
        candidate = mutate_bracket(base_bracket, n_teams)
        if is_valid(candidate, n_teams) && is_unique(candidate, group)
            push!(group, candidate)
        end
    end
    return group
end

function mutate_group(group::Vector{Vector{Vector{Int}}}, n_teams::Int)
    new_group = deepcopy(group)
    idx = rand(1:length(new_group))
    mutated = mutate_bracket(new_group[idx], n_teams)
    if is_valid(mutated, n_teams) && is_unique(mutated, new_group)
        new_group[idx] = mutated
    end
    return new_group
end

function crossover_groups(group1::Vector{Vector{Vector{Int}}}, group2::Vector{Vector{Vector{Int}}}, n_teams::Int)
    point = rand(2:length(group1)-1)
    child1 = vcat(group1[1:point], group2[point+1:end])
    child2 = vcat(group2[1:point], group1[point+1:end])
    child1 = [br for br in child1 if is_valid(br, n_teams)]
    child2 = [br for br in child2 if is_valid(br, n_teams)]
    child1_unique = Vector{Vector{Vector{Int}}}()
    for br in child1
        if is_unique(br, child1_unique)
            push!(child1_unique, br)
        end
    end

    child2_unique = Vector{Vector{Vector{Int}}}()
    for br in child2
        if is_unique(br, child2_unique)
            push!(child2_unique, br)
        end
    end
    while length(child1_unique) < length(group1)
        candidate = mutate_bracket(child1_unique[1], n_teams)
        if is_valid(candidate, n_teams) && is_unique(candidate, child1_unique)
            push!(child1_unique, candidate)
        end
    end
    while length(child2_unique) < length(group2)
        candidate = mutate_bracket(child2_unique[1], n_teams)
        if is_valid(candidate, n_teams) && is_unique(candidate, child2_unique)
            push!(child2_unique, candidate)
        end
    end
    return child1_unique, child2_unique
end


crossover_groups (generic function with 1 method)

In [27]:
function average_max_score_over_simulations_valid(
    group_brackets::Vector{Vector{Vector{Int}}}, 
    n_teams::Int, 
    P::Matrix{Float64}, 
    n_sims::Int=100
)
    for br in group_brackets
        if !is_valid(br, n_teams)
            return -1e6
        end
    end
    total_max_score = 0.0
    for _ in 1:n_sims
        simulated_results = simulate_tournament(n_teams, P)
        scores = [calculate_score(br, simulated_results) for br in group_brackets]
        total_max_score += maximum(scores)
    end
    total_max_score / n_sims
end

function run_ga_groups(
    base_bracket::Vector{Vector{Int}},
    n_teams::Int,
    P::Matrix{Float64},
    pop_size::Int=20,
    generations::Int=30,
    n_sims_per_eval::Int=100,
    group_size::Int=10,
    n_elite::Int=2
)
    population = [initialize_group(base_bracket, n_teams, group_size) for _ in 1:pop_size]
    best_group = nothing
    best_fitness = -Inf
    for gen in 1:generations
        fitnesses = [average_max_score_over_simulations_valid(g, n_teams, P, n_sims_per_eval) for g in population]
        local_max, idx = findmax(fitnesses)
        if local_max > best_fitness
            best_fitness = local_max
            best_group = deepcopy(population[idx])
        end
        println("Generation $gen: Best avg max-score = $local_max")
        sorted_indices = sortperm(fitnesses, rev=true)
        new_population = [deepcopy(population[i]) for i in sorted_indices[1:n_elite]]
        while length(new_population) < pop_size
            i, j = rand(1:pop_size, 2)
            parent1 = fitnesses[i] > fitnesses[j] ? population[i] : population[j]
            i, j = rand(1:pop_size, 2)
            parent2 = fitnesses[i] > fitnesses[j] ? population[i] : population[j]
            child1, child2 = crossover_groups(parent1, parent2, n_teams)
            push!(new_population, mutate_group(child1, n_teams))
            if length(new_population) < pop_size
                push!(new_population, mutate_group(child2, n_teams))
            end
        end
        population = new_population
    end
    return population, best_group, best_fitness
end


run_ga_groups (generic function with 6 methods)

In [28]:
base_bracket = [
    [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
    [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
    [1, 11, 17, 27, 33, 47, 49, 63],
    [1, 17, 33, 49],
    [1, 33],
    [33]
]

pop_size = 20
generations = 30
n_sims_per_eval = 100
group_size = 10
n_elite = 2

population, best_group, best_fitness = run_ga_groups(
    base_bracket, n_teams, P_real, pop_size, generations, n_sims_per_eval, group_size, n_elite
)

println("\nBest group average max-score: $best_fitness\n")


Generation 1: Best avg max-score = 118.25
Generation 2: Best avg max-score = 120.73
Generation 3: Best avg max-score = 120.62
Generation 4: Best avg max-score = 120.99
Generation 5: Best avg max-score = 119.6
Generation 6: Best avg max-score = 120.37
Generation 7: Best avg max-score = 121.0
Generation 8: Best avg max-score = 124.03
Generation 9: Best avg max-score = 123.11
Generation 10: Best avg max-score = 127.0
Generation 11: Best avg max-score = 121.26
Generation 12: Best avg max-score = 124.21
Generation 13: Best avg max-score = 125.47
Generation 14: Best avg max-score = 124.92
Generation 15: Best avg max-score = 120.42
Generation 16: Best avg max-score = 125.94
Generation 17: Best avg max-score = 122.4
Generation 18: Best avg max-score = 122.82
Generation 19: Best avg max-score = 123.91
Generation 20: Best avg max-score = 124.62
Generation 21: Best avg max-score = 127.06
Generation 22: Best avg max-score = 125.15
Generation 23: Best avg max-score = 124.63
Generation 24: Best avg 

In [30]:
brackets = [
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 17, 27, 33, 47, 49, 63],
        [1, 17, 33, 49],
        [1, 33],
        [1]
    ],
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 17, 27, 33, 47, 49, 63],
        [1, 17, 33, 63],
        [1, 33],
        [33]
    ],
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 17, 27, 33, 47, 49, 63],
        [1, 17, 33, 49],
        [17, 33],
        [17]
    ],
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 17, 27, 33, 47, 49, 63],
        [1, 27, 33, 49],
        [1, 49],
        [1]
    ],
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 17, 27, 33, 47, 49, 63],
        [1, 17, 47, 49],
        [1, 47],
        [47]
    ],
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 17, 27, 33, 47, 49, 63],
        [1, 17, 33, 49],
        [1, 49],
        [1]
    ],
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 17, 27, 33, 47, 49, 63],
        [1, 17, 33, 49],
        [17, 49],
        [49]
    ],
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 23, 27, 33, 47, 49, 63],
        [1, 27, 33, 49],
        [1, 33],
        [33]
    ],
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 17, 27, 33, 47, 49, 63],
        [11, 17, 33, 49],
        [11, 33],
        [11]
    ],
    [
        [1, 4, 5, 7, 10, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 36, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63],
        [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63],
        [1, 11, 17, 27, 33, 47, 49, 63],
        [1, 17, 33, 49],
        [17, 33],
        [33]
    ]
]

10-element Vector{Vector{Vector{Int64}}}:
 [[1, 4, 5, 7, 10, 11, 13, 15, 17, 19  …  45, 47, 49, 51, 53, 55, 57, 59, 61, 63], [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63], [1, 11, 17, 27, 33, 47, 49, 63], [1, 17, 33, 49], [1, 33], [1]]
 [[1, 4, 5, 7, 10, 11, 13, 15, 17, 19  …  45, 47, 49, 51, 53, 55, 57, 59, 61, 63], [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63], [1, 11, 17, 27, 33, 47, 49, 63], [1, 17, 33, 63], [1, 33], [33]]
 [[1, 4, 5, 7, 10, 11, 13, 15, 17, 19  …  45, 47, 49, 51, 53, 55, 57, 59, 61, 63], [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63], [1, 11, 17, 27, 33, 47, 49, 63], [1, 17, 33, 49], [17, 33], [17]]
 [[1, 4, 5, 7, 10, 11, 13, 15, 17, 19  …  45, 47, 49, 51, 53, 55, 57, 59, 61, 63], [1, 7, 11, 15, 17, 23, 27, 31, 33, 39, 43, 47, 49, 55, 59, 63], [1, 11, 17, 27, 33, 47, 49, 63], [1, 27, 33, 49], [1, 49], [1]]
 [[1, 4, 5, 7, 10, 11, 13, 15, 17, 19  …  45, 47, 49, 51, 53, 55, 57, 59, 61, 63], [1, 7, 11, 15, 17, 23, 27, 3

In [31]:
for (idx, bracket) in enumerate(brackets)
    if is_valid(bracket, n_teams)
        println("Bracket $idx is valid.")
    else
        println("Bracket $idx is NOT valid.")
    end
end

Bracket 1 is valid.
Bracket 2 is valid.
Bracket 3 is valid.
Bracket 4 is valid.
Bracket 5 is valid.
Bracket 6 is valid.
Bracket 7 is valid.
Bracket 8 is valid.
Bracket 9 is valid.
Bracket 10 is valid.
